In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator
import pickle
import import_ipynb
from normalizing import normalize
#-*- coding: utf-8 -*-
def loading(data_path, eng=True, num=True, punc=False):

    df = pd.read_csv(data_path,sep=",",encoding="utf-8")
    df=np.array(df[:300000])
    
    print(len(df))
    
    title,contents=[],[]
    for doc in df: # doc[0]는 title, doc[1]은 contents
        if type(doc[0]) is not str or type(doc[1]) is not str:
            continue
        if len(doc[0]) > 0 and len(doc[1]) > 0:
            tmptitle = normalize(doc[0], english=eng, number=num, punctuation=punc)
            tmpcontents =  normalize(doc[1], english=eng, number=num, punctuation=punc)
            title.append(tmptitle)
            contents.append(tmpcontents)
    print("title length=",len(title),"content length=",len(contents))
    return title,contents

In [ ]:
def make_dict(contents, minlength, maxlength, jamo_delete=False):
    dict = defaultdict(lambda: [])
    for doc in contents:
        for idx, word in enumerate(doc.split()):
            if len(word) > minlength:
                normalizedword = word[:maxlength]
                if jamo_delete:
                    tmp = []
                    for char in normalizedword:
                        if ord(char) < 12593 or ord(char) > 12643:
                            tmp.append(char)
                    normalizedword = ''.join(char for char in tmp)
                if word not in dict[normalizedword]:
                    dict[normalizedword].append(word)
    dict = sorted(dict.items(), key=operator.itemgetter(0))[1:]
    words = []
    for i in range(len(dict)):
        word = []
        word.append(dict[i][0])
        for w in dict[i][1]:
            if w not in word:
                word.append(w)
        words.append(word)

    words.append(['<PAD>'])
    words.append(['<S>'])
    words.append(['<E>'])
    words.append(['<UNK>'])
    # word_to_ix, ix_to_word 생성
    ix_to_word = {i: ch[0] for i, ch in enumerate(words)}
    word_to_ix = {}
    for idx, words in enumerate(words):
        for word in words:
            word_to_ix[word] = idx
    with open("word_to_ix.pickle", "wb") as f:
            pickle.dump(word_to_ix, f)
    with open("ix_to_word.pickle", "wb") as f:
            pickle.dump(ix_to_word, f)
    
    print("len ix_to_word",len(ix_to_word),"len word_to_ix",len(word_to_ix))
    print('contents number: %s, voca numbers: %s' %(len(contents),len(ix_to_word)))
    
    return word_to_ix,ix_to_word

In [ ]:
def make_suffle(rawinputs, rawtargets, word_to_ix, encoder_size, decoder_size, shuffle=True):
    rawinputs = np.array(rawinputs)
    rawtargets = np.array(rawtargets)
    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(len(rawinputs)))
        rawinputs = rawinputs[shuffle_indices]
        rawtargets = rawtargets[shuffle_indices]
    encoder_input = []
    decoder_input = []
    targets = []
    target_weights = []
    for rawinput, rawtarget in zip(rawinputs, rawtargets):
        tmp_encoder_input = [word_to_ix[v] for idx, v in enumerate(rawinput.split()) if idx < encoder_size and v in word_to_ix]
        encoder_padd_size = max(encoder_size - len(tmp_encoder_input), 0)
        encoder_padd = [word_to_ix['<PAD>']] * encoder_padd_size
        encoder_input.append(list(reversed(tmp_encoder_input + encoder_padd)))
        tmp_decoder_input = [word_to_ix[v] for idx, v in enumerate(rawtarget.split()) if idx < decoder_size - 1 and v in word_to_ix]
        decoder_padd_size = decoder_size - len(tmp_decoder_input) - 1
        decoder_padd = [word_to_ix['<PAD>']] * decoder_padd_size
        decoder_input.append([word_to_ix['<S>']] + tmp_decoder_input + decoder_padd)
        targets.append(tmp_decoder_input + [word_to_ix['<E>']] + decoder_padd)
        tmp_targets_weight = np.ones(decoder_size, dtype=np.float32)
        tmp_targets_weight[-decoder_padd_size:] = 0
        target_weights.append(list(tmp_targets_weight))
    return encoder_input, decoder_input, targets, target_weights

In [ ]:
def doclength(docs,sep=True):
    max_document_length = 0
    for doc in docs:
        if sep:
            words = doc.split()
            document_length = len(words)
        else:
            document_length = len(doc)
        if document_length > max_document_length:
            max_document_length = document_length
    return max_document_length

In [ ]:
def make_batch(encoder_inputs, decoder_inputs, targets, target_weights):
    encoder_size = len(encoder_inputs[0])
    decoder_size = len(decoder_inputs[0])
    encoder_inputs, decoder_inputs, targets, target_weights = np.array(encoder_inputs), np.array(decoder_inputs), np.array(targets), np.array(target_weights)
    result_encoder_inputs = []
    result_decoder_inputs = []
    result_targets = []
    result_target_weights = []
    for i in range(encoder_size):
        result_encoder_inputs.append(encoder_inputs[:, i])
    for j in range(decoder_size):
        result_decoder_inputs.append(decoder_inputs[:, j])
        result_targets.append(targets[:, j])
        result_target_weights.append(target_weights[:, j])
    
    return result_encoder_inputs, result_decoder_inputs, result_targets, result_target_weights

In [ ]:
import tensorflow as tf
import numpy as np
import os
#import import_ipynb
#from utils import loading,make_dict,make_suffle,doclength,make_batch
#from test import load
import time

data_path = './testing_data.csv'
title,content = loading(data_path,eng=False, num=True, punc=False)
word_to_ix,ix_to_word = make_dict(title + content, minlength=0, maxlength=3,jamo_delete=True)


In [ ]:
class seq2seq_attention(object):
    def __init__(self, multi, hidden_size, num_layers, forward_only,learning_rate, batch_size,vocab_size, encoder_size, decoder_size):

        # variables
        self.source_vocab_size = vocab_size
        self.target_vocab_size = vocab_size
        self.batch_size = batch_size
        self.encoder_size = encoder_size
        self.decoder_size = decoder_size
        self.learning_rate = tf.Variable(float(learning_rate), trainable=False)
        self.global_step = tf.Variable(0, trainable=False)

        # networks
        W = tf.Variable(tf.random.normal([hidden_size, vocab_size]))
        #W = tf.Variable(tf.random_normal([hidden_size, vocab_size]))
        #b = tf.Variable(tf.random_normal([vocab_size]))
        b = tf.Variable(tf.random.normal([vocab_size]))
        output_projection = (W, b)
        #tf.placeholder(tf.int32, [batch_size]) 
        self.encoder_inputs = [tf.compat.v1.placeholder(tf.int32, [batch_size]) for _ in range(encoder_size)]  # 인덱스만 있는 데이터 (원핫 인코딩 미시행)
        self.decoder_inputs = [tf.compat.v1.placeholder(tf.int32, [batch_size]) for _ in range(decoder_size)]
        self.targets = [tf.compat.v1.placeholder(tf.int32, [batch_size]) for _ in range(decoder_size)]
        self.target_weights = [tf.compat.v1.placeholder(tf.float32, [batch_size]) for _ in range(decoder_size)]

        # models
        if multi:
            rnn_cells=[]
            #warning two cells provided to MutltiRNNCell are the same object
            for _ in range(num_layers):
                #single_cell = tf.nn.rnn_cell.GRUCell(num_units=hidden_size)
                #tf.compat.v1.nn.rnn_cell.LSTMCell
                single_cell = tf.compat.v1.nn.rnn_cell.GRUCell(num_units=hidden_size)
                rnn_cells.append(single_cell)
                #tf.compat.v1.nn.rnn_cell.MultiRNNCell
            cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell(rnn_cells)
        else:
            cell = tf.compat.v1.nn.rnn_cell.GRUCell(num_units=hidden_size)
            #cell = tf.keras.layers.LSTMCell(units=hidden_size)

        if not forward_only:
            #tf.nn.seq2seq.embedding_attention_seq2seq(
            self.outputs, self.states = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                self.encoder_inputs, self.decoder_inputs, cell,
                num_encoder_symbols=vocab_size,
                num_decoder_symbols=vocab_size,
                embedding_size=hidden_size,
                output_projection=output_projection,
                feed_previous=False)

            self.logits = [tf.matmul(output, output_projection[0]) + output_projection[1] for output in self.outputs]
            self.loss = []
            for logit, target, target_weight in zip(self.logits, self.targets, self.target_weights):
                crossentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=target)
                self.loss.append(crossentropy * target_weight)
            self.cost = tf.add_n(self.loss)#tf.train.AdamOptimizer(learning_rate)
            self.train_op = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(self.cost)


        else:
            self.outputs, self.states = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                self.encoder_inputs, self.decoder_inputs, cell,
                num_encoder_symbols=vocab_size,
                num_decoder_symbols=vocab_size,
                embedding_size=hidden_size,
                output_projection=output_projection,
                feed_previous=True)
            self.logits = [tf.matmul(output, output_projection[0]) + output_projection[1] for output in self.outputs]

    def step(self, session, encoderinputs, decoderinputs, targets, targetweights, forward_only):
        input_feed = {}
        for l in range(len(encoderinputs)):
            input_feed[self.encoder_inputs[l].name] = encoderinputs[l]
            for l in range(len(decoderinputs)):
                input_feed[self.decoder_inputs[l].name] = decoderinputs[l]
                input_feed[self.targets[l].name] = targets[l]
                input_feed[self.target_weights[l].name] = targetweights[l]
        if not forward_only:
            output_feed = [self.train_op, self.cost]
        else:
            output_feed = []
            for l in range(len(decoderinputs)):
                output_feed.append(self.logits[l])
        output = session.run(output_feed,input_feed)
        if not forward_only:
            return output[1] # loss
        else:
            return output[0:] # outputs

In [ ]:
import import_ipynb
import tensorflow as tf
import numpy as np
import time
import os
import pickle

import import_ipynb
from normalizing import normalize
import tensorflow as tf
import numpy as np

def make_3word(content):
    raw_content=""
    for word in content.split():
        if len(word) > 0:
            normalizedword = word[:3]
            tmp = []
            for char in normalizedword:
                if ord(char) < 12593 or ord(char) > 12643:
                    tmp.append(char)
            normalizedword = ''.join(char for char in tmp)
            raw_content += normalizedword+" "
    return raw_content

def make_suffle2(content,word_to_ix,encoder_size):
    tmp=[] 
    tmp.append(content)
    content = np.array(tmp)
    
     
    encoder_input=[]
    
    tmp_encoder_input=[word_to_ix[v] for idx,v in enumerate(content[0].split()) if idx < encoder_size and v in word_to_ix]
    encoder_padd_size=max(encoder_size - len(tmp_encoder_input),0)
    encoder_padd = [word_to_ix['<PAD>']] * encoder_padd_size
    encoder_input.append(list(reversed(tmp_encoder_input+encoder_padd)))

    return encoder_input

In [ ]:
def ml_headline(q2,args1):
     while True:
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        #sess = tf.compat.v1.Session()

        multi = True
        forward_only = False
        hidden_size = 150
        num_layers = 3
        learning_rate = 0.001
        encoder_size = 100

        if not os.path.exists("mm/copy_model"):
            os.mkdir("mm/copy_model")
        else:
            if os.path.exists("mm/copy_model/checkpoint"):
                old_model_checkpoint_path = open('mm/copy_model/checkpoint','r')
                old_model_checkpoint_path = "".join(["model/",old_model_checkpoint_path.read().splitlines()[0].split('"')[1]])

        ########test#########################

        if args1 == "test":
            if old_model_checkpoint_path:

                batch_size = 1
                decoder_size = 19

                with open("ix_to_word.pickle", "rb") as f:
                    ix_to_word = pickle.load(f)
                with open("word_to_ix.pickle", "rb") as t:
                    word_to_ix = pickle.load(t)

                end = 1
                data=q2.get()

                content=normalize(data[1])
                print("content",content)
                content=make_3word(content)
                vocab_size =len(ix_to_word)+1
                encoderinputs=make_suffle2(content,word_to_ix,encoder_size=encoder_size) # padding
                with tf.compat.v1.Session() as sess:

                    print("continuing from previous trained model: ",old_model_checkpoint_path, "...")
                    #saver.restore(sess, old_model_checkpoint_path)
                    model = seq2seq_attention(multi=multi, hidden_size=hidden_size, num_layers=num_layers,
                                learning_rate=learning_rate, batch_size=batch_size,
                                vocab_size=vocab_size,
                                encoder_size=encoder_size, decoder_size=decoder_size,
                                forward_only=True)
                    saver = tf.train.Saver(tf.all_variables())
                    ckpt = tf.train.get_checkpoint_state("./mm/copy_model/")
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    sess.run(tf.all_variables())

                    d=[]
                    #print([0]*15)
                    d.append([0]*15)
                    #print(d)
                    encoder_inputs, decoder_inputs, targets, target_weights = make_batch(encoderinputs[0:end],d,d,d)
                    #print(encoder_inputs)

                    output_logits = model.step(sess,encoder_inputs,decoder_inputs,targets,target_weights,True)
                    predict = [np.argmax(logit,axis=1)[0] for logit in output_logits]
                    predict = ' '.join(ix_to_word[ix] for ix in predict)
                    print(predict)
                    predict=predict.replace("<E>","")
                    data.append(predict)
                    db_update(data)

                tf.reset_default_graph()

        #######train################
        elif args1 == "train":
            batch_size = 16
            decoder_size = doclength(title, sep=True)
            print(decoder_size)
            #decoder_size = util3.doclength(title, sep=True) # (Maximum) number of time steps in this batch
            steps_per_checkpoint = 500

            # transform data
            encoderinputs, decoderinputs, targets_, targetweights = make_suffle(content, title, word_to_ix, encoder_size=encoder_size, decoder_size=decoder_size, shuffle=False)

            with tf.compat.v1.Session(config=config) as sess:
                model = seq2seq_attention(multi=multi, hidden_size=hidden_size, num_layers=num_layers,
                            learning_rate=learning_rate, batch_size=batch_size,
                            vocab_size=vocab_size,
                            encoder_size=encoder_size, decoder_size=decoder_size,
                            forward_only=False)

                sess.run(global_variables_initializer())#global_variables_initializer()
                saver = tf.compat.v1.train.Saver(tf.compat.v1.global_variables())
                step_time, loss = 0.0, 0.0
                current_step = 0
                start = 0
                end = batch_size
                while current_step < 400001:

                    if end > len(title):
                        start = 0
                        end = batch_size

                    # Get a batch and make a step
                    start_time = time.time()
                    encoder_inputs, decoder_inputs, targets, target_weights = make_batch(encoderinputs[start:end],decoderinputs[start:end],targets_[start:end],targetweights[start:end])

                    if current_step % steps_per_checkpoint == 0:
                        for i in range(decoder_size - 2):
                            decoder_inputs[i + 1] = np.array([word_to_ix['<PAD>']] * batch_size)
                        output_logits = model.step(sess, encoder_inputs, decoder_inputs, targets, target_weights, True)
                        predict = [np.argmax(logit, axis=1)[0] for logit in output_logits]
                        predict = ' '.join(ix_to_word[ix] for ix in predict)
                        real = [word[0] for word in targets]
                        real = ' '.join(ix_to_word[ix][0] for ix in real)
                        saver.save(sess, "./mm/copy_model/model.ckpt",global_step=current_step)
                        print('\n----\n step : %s \n time : %s \n LOSS : %s \n prediction : %s \n edit result : %s \n actual result : %s \n----' %
                              (current_step, step_time, loss, predict, real, title[start]))
                        loss, step_time = 0.0, 0.0

                    step_loss = model.step(sess, encoder_inputs, decoder_inputs, targets, target_weights, False)
                    step_time += time.time() - start_time / steps_per_checkpoint
                    loss += np.mean(step_loss) / steps_per_checkpoint
                    current_step += 1
                    start += batch_size
                    end += batch_size


In [ ]:
from lexrankr import LexRank
import random
import requests
import json
from apscheduler.jobstores.base import JobLookupError
from apscheduler.schedulers.background import BackgroundScheduler
import math
import copy
import time
from multiprocessing import Process, Queue
import pandas as pd

#-*- coding: utf-8 -*-
def random_four_news(q1):
    print("random_four_news start")
    articles = requests.get(url="http://34.84.147.192:8000/news/articles/?format=json&limit=1").json()
    count = articles['count']
    print(count)

    articles = requests.get(url="http://34.84.147.192:8000/news/articles/?format=json&limit=" + str(count)).json()
    d = pd.DataFrame(articles['results'])

    df = pd.DataFrame(columns=['news_id', 'summary', 'cluster_id'])
    df = d[['news_id', 'summary', 'cluster_id']]
    cluster = requests.get(url="http://34.84.147.192:8000/news/clusters/").json()
    
    cluster_id = list([i["cluster_id"] for i in cluster if i["cluster_id"] != '07f269a8-3ae6-4994-abfd-e2cb2d4633f3'])
    print(len(cluster_id))
    
    df_summary=[]
    tmp=[]
    i=0
    #먼저 cluster_id에 해당되는 애들 dict에 넣기
    for cluster_id_1 in cluster_id:
        print("x",cluster_id_1)
        df2 = df[df["cluster_id"] == cluster_id_1]
        df_news = df2['news_id'].tolist()
        news_id = random.sample(df_news, 4)
        for news in news_id:
            df3 = df[df["news_id"] == news]
            df_summary += df3['summary'].tolist()
        
        tmp.append(cluster_id_1)
        tmp.append(" ".join(df_summary))
        #print("TTTTTTTtmp",tmp)
        q1.put(tmp)
        tmp=[]
        df_summary=[]



def summary(q1,q2):
    while True:
        print("smry start")
        multi_summary = q1.get()
        
        lexrank = LexRank()
        
        lexrank.summarize(multi_summary[1])  # data (본문)가져와서 요약
        summaries = lexrank.probe(3)  # 3줄요약, summaries 타입은 list
        summaries = '. '.join(summaries)+'.'
        #print("multi-summary= ",summaries)
        multi_summary[1] = summaries
        #ml_headline(data,"test")
        q2.put(multi_summary) # db에 저장되어야 하는 최종 결과
            # 입력데이터가 이상한 값이어서 요약문이 안나올 때 에러처리 #입력데이터가 None으로 들어올때 에러처리
        


def db_update(data):
    URL = "http://34.84.147.192:8000/news/"
    clusterId = data[0]
    summaryy=data[1].split('. ')
    summaryy = "\n\n".join(summaryy)
    print(data)
    cluster = {
        "cluster_headline": data[2],
        "cluster_summary": summaryy
    }
    res = requests.put(url = URL + 'clusters/' + clusterId + '/',data=cluster)


In [ ]:
if __name__ == "__main__":
    q1 = Queue()
    q2 = Queue()
    sched = BackgroundScheduler()

    sched.start()
    sched.add_job(random_four_news, 'cron', minute='57', id="s1", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='12', id="s2", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='15', id="s3", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='18', id="s4", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='21', id="s5", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='24', id="s6", args=[df, cluster_id, q1])
    
    
    process_summary = Process(target=summary, args=(q1,q2,))
    process_mlheadline = Process(target=ml_headline, args=(q2,"test",))
    
    process_four.start()
    process_summary.start()
    process_mlheadline.start()
    
    q1.close()
    q2.close()
    q1.join_thread()
    
    process_summary.join()
    process_mlheadline.join()